In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense


In [2]:
def load_and_preprocess_data(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (64, 64))
        img = img / 255.0   # normalize pixel values between 0 and 1
        images.append(img)
        labels.append(label)
    return images, labels


In [3]:
angry_images, angry_labels = load_and_preprocess_data('dataset/angry', label=0)
fear_images, fear_labels = load_and_preprocess_data('dataset/fear', label=1)
happy_images, happy_labels = load_and_preprocess_data('dataset/happy', label=2)
neutral_images, neutral_labels = load_and_preprocess_data('dataset/neutral', label=3)
sad_images, sad_labels = load_and_preprocess_data('dataset/sad', label=4)
surprise_images, surprise_labels = load_and_preprocess_data('dataset/surprise', label=5)


In [4]:
all_images = np.concatenate([angry_images, fear_images, happy_images, neutral_images, sad_images, surprise_images])
all_labels = np.concatenate([angry_labels, fear_labels, happy_labels, neutral_labels, sad_labels, surprise_labels])


In [5]:
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)


In [6]:
# Load the ResNet50 model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
# Add your custom dense layers on top of the pre-trained model
res_model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dense(6, activation='softmax')  # Assuming you have 6 classes
])

In [7]:

batch_size = 64
nb_epochs = 20


In [8]:
# Add callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)


In [9]:
# Compile and train the model with callbacks
res_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
res_model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epochs, verbose=1,
              validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr])

Epoch 1/20
266/266 [==============================] - 651s 2s/step - loss: 1.4359 - accuracy: 0.4394 - val_loss: 1.8607 - val_accuracy: 0.1723 - lr: 0.0010
Epoch 2/20
266/266 [==============================] - 633s 2s/step - loss: 1.2975 - accuracy: 0.4925 - val_loss: 1.8065 - val_accuracy: 0.1756 - lr: 0.0010
Epoch 3/20
266/266 [==============================] - 658s 2s/step - loss: 1.2541 - accuracy: 0.5073 - val_loss: 1.7723 - val_accuracy: 0.2145 - lr: 0.0010
Epoch 4/20
266/266 [==============================] - 655s 2s/step - loss: 1.1746 - accuracy: 0.5438 - val_loss: 1.5964 - val_accuracy: 0.3435 - lr: 0.0010
Epoch 5/20
266/266 [==============================] - 653s 2s/step - loss: 1.1270 - accuracy: 0.5619 - val_loss: 1.2324 - val_accuracy: 0.5203 - lr: 0.0010
Epoch 6/20
266/266 [==============================] - 637s 2s/step - loss: 1.0135 - accuracy: 0.6114 - val_loss: 1.4127 - val_accuracy: 0.4552 - lr: 0.0010
Epoch 7/20
266/266 [==============================] - 638s 2s/st

In [10]:
score = res_model.evaluate(X_test, y_test,verbose=0 )
print("Test Score: ", score[0])
print("Test accuracy: ", score[1])

Test Score:  1.1789156198501587
Test accuracy:  0.5843941569328308


In [11]:
res_model.save("newmodel_res.h5")

C:\Users\91790\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
from tensorflow.keras.models import load_model
loaded_model=load_model("newmodel_res.h5")

In [13]:
# Function to preprocess a test image
def preprocess_test_image(image_path):
    img = cv2.imread(image_path)
    if img is not None:
        img = cv2.resize(img, (64, 64))
        img = img / 255.0
        return img
    else:
        return None

In [14]:
class_labels = {
        0: 'anger',
        1: 'fear',
        2: 'happy',
        3: 'neutral',
        4: 'sad',
        5: 'surprise'
    }


In [15]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model



In [ ]:
emotion_model = load_model("newmodel_res.h5")

# Create a cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open a connection to the webcam (0 represents the default camera)
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
     # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    # Iterate over detected faces
    for (x, y, w, h) in faces:
        # Extract the face region
        face_roi = frame[y:y + h, x:x + w]

        # Preprocess the face image for emotion prediction
        face_roi = cv2.resize(face_roi, (64, 64))
        face_roi = face_roi / 255.0
        face_roi = np.expand_dims(face_roi, axis=0)

        # Make emotion prediction
        emotion_prediction = emotion_model.predict(face_roi)
        predicted_emotion = class_labels[np.argmax(emotion_prediction)]
        confidence = np.max(emotion_prediction)

        # Display the predicted emotion on the frame
        cv2.putText(frame, f"Emotion: {predicted_emotion} ({confidence:.2f})", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Draw a rectangle around the detected face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Display the frame
    cv2.imshow('Emotion Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break



1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 42ms/step


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 34ms/step


...

1/1 [==============================] - 0s 74ms/step


1/1 [==============================] - 0s 98ms/step


1/1 [==============================] - 0s 90ms/step


1/1 [==============================] - 0s 104ms/step


1/1 [==============================] - 0s 77ms/step
